In [86]:
knobs2 = {
#     "appendonly": None,
#     "appendfsync": None,
#     "auto-aof-rewrite-percentage": None,
#     "auto-aof-rewrite-min-size": None,
#     "no-appendfsync-on-rewrite": None,
#     "aof-rewrite-incremental-fsync": None,
#     "aof-use-rdb-preamble": None,

    # "save": None,
    "rdbcompression": None,
    "rdbchecksum": None,
    "rdb-save-incremental-fsync": None,

    "activedefrag": None,
    "active-defrag-threshold-lower": None,
    "active-defrag-threshold-upper": None,
    "active-defrag-cycle-min": None,
    "active-defrag-cycle-max": None,

    "maxmemory": None,
    "maxmemory-policy": None,
    "maxmemory-samples": None,
    "lazyfree-lazy-eviction": None,
    "lazyfree-lazy-expire": None,
    "lazyfree-lazy-server-del": None,

    "hash-max-ziplist-entries": None,
    "hash-max-ziplist-value": None,
    "activerehashing": None,
    "hz": None,
    "dynamic-hz": None
}

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import xgboost
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from lightgbm import LGBMRegressor, plot_importance
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor

## local setting

In [23]:
# config data(.csv) -> DataFrame
config_df = pd.read_csv("./result_config10001_11000.csv", sep=',')

In [24]:
config_df

,appendonly,appendfsync,auto-aof-rewrite-percentage,auto-aof-rewrite-min-size,no-appendfsync-on-rewrite,aof-rewrite-incremental-fsync,aof-use-rdb-preamble,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,...,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,...,volatile-lfu,7,no,yes,no,636,235,yes,7,no
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,...,volatile-random,3,no,yes,yes,392,67,yes,21,yes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,yes,...,volatile-lfu,7,yes,yes,yes,699,94,yes,37,yes
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,yes,no,...,volatile-lfu,7,yes,no,yes,400,136,no,9,no
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,...,volatile-lru,7,yes,no,no,663,218,no,12,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,yes,...,allkeys-lfu,3,yes,yes,no,626,149,yes,13,no
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,...,volatile-random,6,yes,yes,yes,745,129,yes,31,yes
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,no,...,volatile-lfu,6,yes,yes,yes,302,101,no,12,yes
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,yes,...,volatile-lfu,4,no,yes,yes,291,67,yes,19,yes


In [25]:
# rdb knob으로 분류
config_df_rdb = config_df.loc[:,'rdbcompression':'dynamic-hz']

In [26]:
config_df_rdb.head()

,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,activedefrag,active-defrag-threshold-lower,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
0,no,no,no,yes,7.0,83.0,12.0,83.0,NaN,volatile-lfu,7,no,yes,no,636,235,yes,7,no
1,no,no,no,NaN,NaN,NaN,NaN,NaN,NaN,volatile-random,3,no,yes,yes,392,67,yes,21,yes
2,no,no,yes,yes,28.0,93.0,28.0,86.0,NaN,volatile-lfu,7,yes,yes,yes,699,94,yes,37,yes
3,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,volatile-lfu,7,yes,no,yes,400,136,no,9,no
4,yes,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,volatile-lru,7,yes,no,no,663,218,no,12,no


In [27]:
# external data(.csv) -> DataFrame
external_df = pd.read_csv("./result_rdb_external_4.csv", sep=',')

In [28]:
# throuput 값으로 예상
external_df['Gets_KB/sec']

0      68121.32
1      67585.45
2      67425.93
3      67046.94
4      66755.71
         ...   
995    67567.17
996    68655.48
997    67605.50
998    67812.35
999    66795.58
Name: Gets_KB/sec, Length: 1000, dtype: float64

## 데이터 전처리

In [29]:
categorical_var = ['rdbcompression', 'rdbchecksum','rdb-save-incremental-fsync', 'activedefrag', 'maxmemory-policy','lazyfree-lazy-eviction','lazyfree-lazy-expire','lazyfree-lazy-server-del','activerehashing', 'dynamic-hz']

for cate_var in categorical_var:
    config_df_rdb[cate_var] = config_df_rdb[cate_var].astype('category') 


In [30]:
config_data = config_df_rdb
throughput_data = external_df['Gets_KB/sec']

In [32]:
config_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   rdbcompression                 1000 non-null   category
 1   rdbchecksum                    1000 non-null   category
 2   rdb-save-incremental-fsync     1000 non-null   category
 3   activedefrag                   510 non-null    category
 4   active-defrag-threshold-lower  510 non-null    float64 
 5   active-defrag-threshold-upper  510 non-null    float64 
 6   active-defrag-cycle-min        510 non-null    float64 
 7   active-defrag-cycle-max        510 non-null    float64 
 8   maxmemory                      0 non-null      float64 
 9   maxmemory-policy               1000 non-null   category
 10  maxmemory-samples              1000 non-null   int64   
 11  lazyfree-lazy-eviction         1000 non-null   category
 12  lazyfree-lazy-expire           1000

In [33]:
# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(config_data, throughput_data ,test_size=0.2)


In [77]:
fit_params={"early_stopping_rounds":10, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': 'auto' # that's actually the default
           }

In [78]:
lgbr = lgb.LGBMRegressor(num_leaves= 15, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, 
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.1)


In [79]:
lgbr.fit(X_train, y_train, **fit_params)

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid's auc: 1


/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


LGBMRegressor(colsample_bytree=0.9, metric='None', n_estimators=1000, n_jobs=4,
              num_leaves=15, random_state=314, subsample=0.9)

In [81]:
y_predict = lgbr.predict(X_test)

In [82]:
X_test.head()

,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,activedefrag,active-defrag-threshold-lower,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
915,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,volatile-ttl,3,no,no,yes,394,47,yes,13,no
801,no,yes,yes,yes,7.0,85.0,3.0,85.0,NaN,noeviction,5,no,yes,yes,495,39,yes,29,yes
322,no,no,no,yes,24.0,93.0,15.0,71.0,NaN,noeviction,5,no,yes,no,458,64,yes,19,no
529,yes,no,no,NaN,NaN,NaN,NaN,NaN,NaN,volatile-random,6,yes,no,yes,459,105,yes,31,no
449,no,no,yes,NaN,NaN,NaN,NaN,NaN,NaN,volatile-ttl,6,yes,yes,yes,694,171,no,17,no


In [85]:
X_test.iloc[0:2,]
clf.predict(X_test.iloc[0:1,])

X_test.iloc[0:1,]



,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,activedefrag,active-defrag-threshold-lower,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
915,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,volatile-ttl,3,no,no,yes,394,47,yes,13,no


In [84]:
y_predict

array([67538.44646868, 67529.29893427, 67519.24807113, 67549.71824425,
       67540.30839048, 67504.41024114, 67504.41024114, 67575.27564241,
       67522.59117024, 67540.30839048, 67538.44646868, 67538.44646868,
       67540.30839048, 67538.44646868, 67504.41024114, 67504.41024114,
       67578.72250462, 67465.71080886, 67538.44646868, 67540.30839048,
       67504.41024114, 67504.41024114, 67504.41024114, 67538.44646868,
       67504.35170512, 67504.41024114, 67540.30839048, 67578.72250462,
       67519.24807113, 67549.71824425, 67504.41024114, 67522.59117024,
       67549.71824425, 67578.72250462, 67504.41024114, 67565.51173569,
       67519.24807113, 67578.72250462, 67568.57079982, 67489.33073074,
       67533.65572614, 67522.59117024, 67578.72250462, 67504.41024114,
       67578.72250462, 67504.41024114, 67540.30839048, 67504.41024114,
       67549.71824425, 67504.41024114, 67504.41024114, 67504.41024114,
       67540.30839048, 67549.71824425, 67504.41024114, 67522.59117024,
      